In [1]:
# importer les packages
import numpy as np
import matplotlib 
import pandas as pd
import duckdb as db

In [18]:
# importer les bases de données

orders = pd.read_csv("C:/Users/Moi/sql_challenge/database/orders.csv")

order_items = pd.read_csv("C:/Users/Moi/sql_challenge/database/order_items.csv")

products = pd.read_csv("C:/Users/Moi/sql_challenge/database/products.csv")

orders.head(3)

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1


In [20]:
products.head(3)

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1


In [6]:
order_items.head(3)

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02


In [12]:
# Une requête qui attribue un numéro de rang à chaque élément de commande en fonction 
# du prix de vente décroissant et les ordonne par prix de vente décroissant.

query = """ 
SELECT
   *,
    ROW_NUMBER() OVER (ORDER BY sale_price DESC) AS price_rank
FROM
    order_items
ORDER BY
    sale_price DESC;


"""
db.sql(query)

┌────────┬──────────┬─────────┬────────────┬───┬──────────────────────┬────────────────────┬────────────┐
│   id   │ order_id │ user_id │ product_id │ … │     returned_at      │     sale_price     │ price_rank │
│ int64  │  int64   │  int64  │   int64    │   │       varchar        │       double       │   int64    │
├────────┼──────────┼─────────┼────────────┼───┼──────────────────────┼────────────────────┼────────────┤
│ 114349 │    78780 │   62841 │      23546 │ … │ NULL                 │              999.0 │          1 │
│ 149459 │   102920 │   82176 │      23546 │ … │ NULL                 │              999.0 │          2 │
│  37188 │    25610 │   20434 │      24447 │ … │ NULL                 │              999.0 │          3 │
│  39329 │    27087 │   21671 │      24447 │ … │ NULL                 │              999.0 │          4 │
│  41201 │    28344 │   22656 │      23546 │ … │ NULL                 │              999.0 │          5 │
│  35807 │    24643 │   19650 │      24447 │ …

In [36]:
# Une requête qui compte le nombre de commandes par utilisateur, attribue un numéro de rang aux utilisateurs en fonction 
# du nombre de commandes décroissant
query = """ 
SELECT
    user_id,
    COUNT(order_id) AS num_orders,
    ROW_NUMBER() OVER (ORDER BY COUNT(order_id) DESC) AS user_order_rank
FROM
    order_items
GROUP BY
    user_id




"""
db.sql(query)

┌─────────┬────────────┬─────────────────┐
│ user_id │ num_orders │ user_order_rank │
│  int64  │   int64    │      int64      │
├─────────┼────────────┼─────────────────┤
│   32996 │         14 │               1 │
│   80546 │         13 │               2 │
│   80952 │         13 │               3 │
│   52039 │         12 │               4 │
│   34249 │         12 │               5 │
│   21909 │         12 │               6 │
│   40896 │         12 │               7 │
│   98947 │         12 │               8 │
│   46712 │         12 │               9 │
│   19415 │         12 │              10 │
│     ·   │          · │               · │
│     ·   │          · │               · │
│     ·   │          · │               · │
│   25920 │          4 │            9991 │
│   29074 │          4 │            9992 │
│   43632 │          4 │            9993 │
│   48287 │          4 │            9994 │
│   50623 │          4 │            9995 │
│   58591 │          4 │            9996 │
│   78834 │

In [27]:
#Cette requête  attribue un numéro de rang à chaque produit dans chaque catégorie, en les ordonnant par prix de détail décroissant 
# dans chaque catégorie. 
query = """ 
SELECT
    id AS product_id,
    category,
    name,
    brand,
    retail_price,
    ROW_NUMBER() OVER (PARTITION BY category ORDER BY retail_price DESC) AS price_rank_per_category
FROM
    products;


"""
db.sql(query)

┌────────────┬──────────┬──────────────────────┬────────────────────────┬────────────────────┬─────────────────────────┐
│ product_id │ category │         name         │         brand          │    retail_price    │ price_rank_per_category │
│   int64    │ varchar  │       varchar        │        varchar         │       double       │          int64          │
├────────────┼──────────┼──────────────────────┼────────────────────────┼────────────────────┼─────────────────────────┤
│      15486 │ Plus     │ NIKE WOMEN'S PRO C…  │ Nike                   │              903.0 │                       1 │
│      15956 │ Plus     │ BGSD Women's New Z…  │ BGSD                   │   319.989990234375 │                       2 │
│      15354 │ Plus     │ Bali Women's One S…  │ Bali                   │  316.6000061035156 │                       3 │
│      15898 │ Plus     │ Oakley Men's Racin…  │ Oakley                 │              280.0 │                       4 │
│      15512 │ Plus     │ DOLCE 

In [35]:
# Cette requête attribue un numéro de rang à chaque produit dans chaque catégorie.
query = """ 
SELECT
    id AS product_id,
    category,
    name,
    brand,
    retail_price,
    ROW_NUMBER() OVER (PARTITION BY category) AS price_rank_per_category
FROM
    products;

"""
db.sql(query)

┌────────────┬─────────────────────┬──────────────────────┬───┬────────────────────┬──────────────────────┐
│ product_id │      category       │         name         │ … │    retail_price    │ price_rank_per_cat…  │
│   int64    │       varchar       │       varchar        │   │       double       │        int64         │
├────────────┼─────────────────────┼──────────────────────┼───┼────────────────────┼──────────────────────┤
│      15512 │ Plus                │ DOLCE & GABBANA DG…  │ … │              243.0 │                    1 │
│      15426 │ Plus                │ Gold Toe Women's F…  │ … │               15.0 │                    2 │
│      15408 │ Plus                │ Shadowline Short S…  │ … │               36.0 │                    3 │
│      15460 │ Plus                │ Shadowline Silhoue…  │ … │              104.0 │                    4 │
│      15448 │ Plus                │ ASICS Unisex Adult…  │ … │               22.0 │                    5 │
│      15273 │ Plus         